In [1]:
import os, pickle, random, time

from tqdm.notebook import tqdm

import numpy as np
from PIL import Image

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.utils import save_image

from BBNN import *

D:\Desktop\FIRMsDigitizing\RECTDNN\BBNN.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
base_dir = r"C:\Users\franc\OneDrive - University Of Houston\AAA_RECTDNN\data\BBNN\tileindices\\"

image_dir = f"{base_dir}in\\"
pred_dir  = f"{base_dir}pkl\\"

colorjitter = T.ColorJitter(0.5, 0.2, 0.2, 0.2)
colorjitter.saturation=None
colorjitter.contrast=None

transform = RandomApplyToList(torch.nn.Sequential(
     T.Resize(2048),
     T.RandomRotation(50),
     colorjitter
))

dataset = BBNN_dataset(image_dir, pred_dir, transform=transform)
# loader  = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)

C:\Users\franc\anaconda3\envs\yolo\lib\site-packages\torchvision\transforms\functional.py:153: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:212.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()
C:\Users\franc\anaconda3\envs\yolo\lib\site-packages\PIL\Image.py:3157: DecompressionBombWarning: Image size (149040000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
C:\Users\franc\anaconda3\envs\yolo\lib\site-packages\PIL\Image.py:3157: DecompressionBombWarning: Image size (149238064 pixels) exceeds limit of 89478485 pixels, could be decomp

In [3]:
if False:
    for x in range(len(dataset)):
        fig, axs = plt.subplots(1, 4, figsize=(15, 4))
        axs[0].imshow(np.asarray(dataset.data[x]['image']))
        for i in range(3):
            if len(dataset.data[x]['masks']) <= i:
                continue
            axs[i+1].imshow(dataset.data[x]['masks'][i].numpy()[0])
    
        plt.show()

In [4]:
aug_dir = r"C:\Users\franc\OneDrive - University Of Houston\AAA_RECTDNN\data\BBNN\tileindices\augmentv4\\"

imdir  = f"{aug_dir}images"
mdir   = f"{aug_dir}masks"
pkldir = f"{aug_dir}pkl"

for ii in tqdm(range(100)):
    for i in range(len(dataset)):
        curr_data = dataset[i]

        while curr_data['predictions']['boxes'].sum() == 0:
            curr_data = dataset[i]

        for iii in range(3):
            if len(curr_data['masks']) <= i:
                continue
                
            Image.fromarray((curr_data['masks'][iii].numpy()[0] * 255).astype(np.uint8)).save(os.path.join(mdir, f'{ii}_{i}_{iii}.png'))
        
        save_image(curr_data['image'], os.path.join(imdir, f'{ii}_{i}.png'))
        
        out_dict = {
            'labels' : curr_data['predictions']['labels'],
            'boxes'  : curr_data['predictions']['boxes'].numpy().tolist()
        }
        f = open(os.path.join(pkldir, f'{ii}_{i}.pkl'), 'wb')
        pickle.dump(out_dict,f)
        f.close()

  0%|          | 0/100 [00:00<?, ?it/s]